## Redshift elastic resize demo

* This demo shows how elastic resize can scale redshift cluster up or down with minimum impact for your workload
    - Step 1, use query runner to simulate a continous workload
    - Step 2, initiate elastic resize from AWS console, increase cluster size to 2x
    - Step 3, start monitoring in notebook
    - Step 4, Once resize is done, observe workload continuous, total cluster downtime is only a few minutes
    - Step 5, Run the IO-heavy workload (like ETL), observe the performance is better than the original cluster
    - Step 6, Show the number of slices is not changed after elastic resize


In [29]:
%%time
import boto3
import json
import time

iamconnectioninfo = IamConnection()
cluster_info = iamconnectioninfo.hostname_plus_port.split('.')
clusterid = cluster_info[0]
region = cluster_info[2]

boto3.setup_default_session()
client = boto3.client(service_name='redshift', region_name=region)

start_time = time.time()
print(time.ctime())

# Monitor resizing status, check every 30 seconds
for i in range(10*60):
    response = client.describe_resize(
        ClusterIdentifier=clusterid
    )
    print(response['Status'])
    if response['Status'] == 'SUCCEEDED':
        print('%s is completed successful at %s.' % (response['ResizeType'], time.ctime()))
        print((time.time() - start_time)/60)
        break
    time.sleep(30)


Thu Aug  1 16:19:49 2019


ResizeNotFoundFault: An error occurred (ResizeNotFound) when calling the DescribeResize operation: No Resize found for Cluster rgs-89adc4b1-rvt4tqch

Below code shows how to do elastic resize programmatically, so you can create a lambda function and schedule reoccurance, 
* e.g. sacle up for nightly ETL jobs, and scale down for day job 

In [ ]:
# Send resize request. Note
# 1.5-2 minutes to size down
# but 5-10 minutes to size up
boto3.setup_default_session()
client = boto3.client(service_name='redshift', region_name=region)

response = client.resize_cluster(
    ClusterIdentifier=clusterid,
    ClusterType='multi-node',
    NodeType='dc2.large',
    NumberOfNodes=4,
    Classic=False
)
print(response['Cluster']['ClusterStatus'])


In [30]:
%%time
import boto3
import sh
import os

stack_name_path = '/home/ec2-user/SageMaker/assets/metadata/stackname.txt'

session = boto3.session.Session()
client = session.client("redshift")
stack_name = open(stack_name_path, 'r').readline().rstrip('\n')
response = client.describe_clusters(TagKeys=['stack_name'],
                                    TagValues=[stack_name])
print(response['Clusters'][0]['Endpoint']['Address'])
#print(response['Clusters'][0])


rgs-89adc4b1-rvt4tqch.cvm00saj2qsp.us-east-1.redshift.amazonaws.com
CPU times: user 48.3 ms, sys: 3.28 ms, total: 51.5 ms
Wall time: 137 ms
